In [1]:
import numpy as np
import pandas as pd
import nltk
import re

In [2]:
data = pd.read_csv('dataafterfilternew.csv')#Read dataafterfilter.csv

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
count_vec = CountVectorizer()
# cross validation
#x1,y1=data.iloc[:3000,3].fillna(' '),data.iloc[:3000,1]
#x2,y2=data.iloc[6001:,3].fillna(' '),data.iloc[6001:,3]
#x_train,y_train=x1.append(x2),y1.append(y2)
x_train, y_train = data.iloc[3000:, 3].fillna(' ') , data.iloc[3000:, 1]#All words after 3000 words for train，x_train is content，y_train is class
x_test, y_test = data.iloc[:3000, 3].fillna(' '), data.iloc[:3000, 1]#Top 3000 words for test
# word count by CountVectorizer
x_train_mnb = count_vec.fit_transform(x_train)
x_test_mnb = count_vec.transform(x_test)

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer#Here is the classification using the tf-idf method
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
# calculate the tfidf of the text 
tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train_mnb)
x_test_tf = tfidf.transform(x_test_mnb)

#predict by native bayes
mnb = MultinomialNB()
mnb.fit(x_train_tf, y_train)
print(classification_report(y_test, mnb.predict(x_test_tf)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        84
          1       0.65      0.96      0.77      1683
          2       0.72      0.48      0.58       545
          3       0.67      0.15      0.24       688

avg / total       0.65      0.66      0.59      3000



G:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
from sklearn import svm  #Here is the classification using the svm method
from sklearn.linear_model import SGDClassifier
#predict by SVM
dtc = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)

dtc.fit(x_train_mnb, y_train)
print(classification_report(y_test, dtc.predict(x_test_mnb)))

             precision    recall  f1-score   support

          0       0.92      0.13      0.23        84
          1       0.73      0.96      0.83      1683
          2       0.78      0.64      0.71       545
          3       0.81      0.40      0.54       688

avg / total       0.76      0.75      0.72      3000



G:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [6]:
from sklearn.ensemble import GradientBoostingClassifier#Here is the classification using the gradient boosting method
gbdt=GradientBoostingClassifier(n_estimators=200) 

#predict by GBDT
gbdt.fit(x_train_tf, y_train)
print(classification_report(y_test, gbdt.predict(x_test_tf.toarray())))

             precision    recall  f1-score   support

          0       0.53      0.37      0.44        84
          1       0.63      0.97      0.77      1683
          2       0.88      0.44      0.59       545
          3       0.75      0.08      0.14       688

avg / total       0.70      0.65      0.58      3000



In [7]:
data['isEqual'] = dtc.predict(x_test_mnb) == y_test

In [8]:
result = data.iloc[:3000]#The first 3000 classified data are intercepted as result

In [9]:
result.head(3)

,tweet_id,sentiment,author,content,isEqual
0,1956967341,0,xoshayzers,know listenin bad habit earlier started freaki...,False
1,1956967666,1,wannamama,n bed headache,True
2,1956967696,3,coolfunky,NaN,True


In [10]:
result['predict'] = dtc.predict(x_test_mnb)#Here is the result of using the svm method, because the correct rate (f1-score) obtained by the SVM method is always the highest

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
result.head(3)

,tweet_id,sentiment,author,content,isEqual,predict
0,1956967341,0,xoshayzers,know listenin bad habit earlier started freaki...,False,1
1,1956967666,1,wannamama,n bed headache,True,1
2,1956967696,3,coolfunky,NaN,True,3
